[Rendered Notebook](http://nbviewer.jupyter.org/github/andersonfrailey/Notebook-Uploads/blob/master/TaxCalc%20Benefit%20Functionality.ipynb)

In [1]:
from taxcalc import *
from bokeh.plotting import figure
from bokeh.io import show, output_notebook, push_notebook
from bokeh.models import HoverTool, ColumnDataSource, NumeralTickFormatter, Range1d, DataRange1d
from bokeh.models.widgets import Select
from bokeh.models.annotations import LabelSet
from bokeh.layouts import row, column, gridplot
from notebookfunctions import distribution, percentile, weighted_sum
import pandas as pd
import numpy as np
import copy
output_notebook()

Loading BokehJS ...

In [2]:
start_year = 2014
end_year = 2027
years = [i for i in range(start_year, end_year + 1)]

### Functions for benefits plots

In [3]:
# function for participation rate by program and year
def participation_rates_year(year, cds):
    part_fig = figure(title='Participation Rates by AGI Percentile for {}'.format(year))
    mcare_part = part_fig.line(x='index', y='Medicare', legend='Medicare ', line_width=2,
                               source=cds)
    mcaid_part = part_fig.line(x='index', y='Medicaid', legend='Medicaid ', line_width=2, color='red',
                               source=cds)
    ss_part = part_fig.line(x='index', y='Social Security', legend='Social Security ', line_width=2, color='green',
                            source=cds)
    snap_part = part_fig.line(x='index', y='SNAP', legend='SNAP ', line_width=2, color='orange',
                              source=cds)
    ssi_part = part_fig.line(x='index', y='SSI', legend='SSI ', line_width=2, color='purple',
                             source=cds)
    vb_part = part_fig.line(x='index', y='Veterans Benefits', legend='Veterans Benefits ', line_width=2, color='pink',
                            source=cds)
    hover_part = HoverTool(tooltips=[('Rate', '$y{0.0%}'), ('Percentile', '@index%')])
    part_fig.add_tools(hover_part)

    part_fig.yaxis.axis_label = 'Rate'
    part_fig.xaxis.axis_label = 'AGI Percentile'
    
    return part_fig

In [4]:
# function for average benefits
def avg_ben(title, cds):
    fig = figure(title=title)
    fig.line(x='index', y='Medicare', legend='Medicare ', line_width=2,
             source=cds)
    fig.line(x='index', y='Medicaid', legend='Medicaid ', line_width=2, color='red',
             source=cds)
    fig.line(x='index', y='Social Security', legend='Social Security ', line_width=2, color='green',
             source=cds)
    fig.line(x='index', y='SNAP', legend='SNAP ', line_width=2, color='orange',
             source=cds)
    fig.line(x='index', y='SSI', legend='SSI ', line_width=2, color='purple',
             source=cds)
    fig.line(x='index', y='Veterans Benefits', legend='Veterans Benefits ', line_width=2, color='pink',
             source=cds)
    hover_ab = HoverTool(tooltips=[('Benefit', '$y{$0,0.00}'), ('Percentile', '@index%')])
    fig.add_tools(hover_ab)

    fig.xaxis.axis_label = 'Percentile'
    fig.yaxis.axis_label = 'Average Benefit'
    fig.legend.orientation = 'horizontal'
    fig.legend.location = 'top_center'
    fig.width = 800
    
    return fig

In [5]:
def total_bar(data, ben):
    """
    """
    f = figure(title='Total {} Benefits by Year'.format(ben),
               x_range=DataRange1d())
    bar = f.vbar(x='years', top='total', width=0.75, bottom=0, source=data)
    hover_format = '{$ 0.00 a}'
    hover = HoverTool(tooltips=[('Value', '@total{$0.00 a}')])
    f.add_tools(hover)
    f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
    f.xaxis.minor_tick_line_color = None
    f.xaxis.axis_label = 'Year'
    f.yaxis.axis_label = 'Total Benefit'
    return f

# Sample Reform

In [6]:
# baseline calculator
pol = Policy()
rec = Records.cps_constructor()
calc = Calculator(policy=pol, records=rec)
calc.advance_to_year(2018)
calc.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


## Repeal SSI, SNAP, Other Benefits

In [7]:
# reform calculator
pol2 = Policy()
reform = {
    2018: {'_BEN_Benefit_switch': [[False, False, True, True, True, True, False]]}
}
pol2.implement_reform(reform)
rec2 = Records.cps_constructor()
calc2 = Calculator(policy=pol2, records=rec2)
calc2.advance_to_year(2018)
calc2.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [8]:
baseline_df = calc.dataframe(['ben_total', 's006'])
reform_df = calc2.dataframe(['ben_total', 's006'])
baseline_benefits = (baseline_df.ben_total * baseline_df.s006).sum()
reform_benefits = (reform_df.ben_total * reform_df.s006).sum()
change = reform_benefits - baseline_benefits
print ('Baseline Benefits: {:18.2f}'.format(baseline_benefits))
print ('Reform Benefits: {:20.2f}'.format(reform_benefits))
print ('-' * 37)
print ('Change in Benefits: {:17.2f}'.format(change))

Baseline Benefits:   2455611346834.01
Reform Benefits:     2319481821292.28
-------------------------------------
Change in Benefits:  -136129525541.73


# Benefit Plots

In [9]:
records = Records.cps_constructor()
policy = Policy()
calc = Calculator(records=records, policy=policy)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [10]:
# list of variables to pull from calculator
wanted_vars = ['s006', 'c00100', 'snap_ben', 'ssi_ben', 'oasdi_ben', 'vet_ben',
               'mcare_ben', 'mcaid_ben']
# list of benefits
ben_list = ['snap', 'ssi', 'oasdi', 'vet', 'mcare', 'mcaid']
ben_name_map = {'snap': 'SNAP', 'ssi': 'SSI', 'oasdi': 'Social Security',
                'vet': 'Veterans Benefits', 'mcare': 'Medicare', 'mcaid': 'Medicaid'}

In [11]:
# dictionary to hold all of the dataframes for each year
var_dict = {}
for year in range(start_year, end_year + 1):
    # advance to the year
    calc.advance_to_year(year)
    calc.calc_all()
    # pull wanted variables
    var_df = calc.dataframe(wanted_vars)
    # add participation indicators for each variable
    for ben in ben_list:
        ben_name = '{}_ben'.format(ben)
        part_name = '{}_part'.format(ben)
        var_df[part_name] = np.where(var_df[ben_name] > 0, 1, 0)
    # add the DataFrames to the dictionary
    var_dict[year] = var_df

In [12]:
total_bar_list = []
# find the total benefit for each year
for ben in ben_list:
    total_dict = {'total': []}
    ben_name = '{}_ben'.format(ben)
    for year in range(start_year, end_year + 1):
        year_df = var_dict[year]
        total_dict['total'].append((year_df[ben_name] * year_df['s006']).sum())
    total_dict['years'] = years
    total_cds = ColumnDataSource(pd.DataFrame(total_dict))
    b_name = ben_name_map[ben]
    fig = total_bar(total_cds, b_name)
    total_bar_list.append(fig)

In [13]:
show(column(total_bar_list))

# Participation Rates by AGI Percentile

In [14]:
part_list = []
for year in years:
    data = var_dict[year]
    data_dict = {'index': [i for i in range(1, 101)]}
    for ben in ben_list:
        part_name = '{}_part'.format(ben)
        # find the participation rates
        p_rate = percentile(data, part_name, 100, 'c00100')
        data_dict[ben_name_map[ben]] = p_rate
    # create CDS
    cds = ColumnDataSource(data_dict)
    # create plot
    plot = participation_rates_year(year, cds)
    part_list.append(plot)

In [15]:
show(column(part_list))

# Average benefits for All

In [16]:
avg_all_list = []
for year in years:
    title = 'Average Benefit for Participants by AGI Percentile for {}'.format(year)
    data = var_dict[year]
    data_dict = {'index': [i for i in range(1, 101)]}
    for ben in ben_list:
        ben_name = '{}_ben'.format(ben)
        # find the average benefits
        ben_avg = percentile(data, ben_name, 100, 'c00100')
        data_dict[ben_name_map[ben]] = ben_avg
    # create CDS
    cds = ColumnDataSource(data_dict)
    # create plot
    plot = avg_ben(title, cds)
    avg_all_list.append(plot)

In [17]:
show(column(avg_all_list))

# Average Benefits for Participants

In [18]:
avg_part_list = []
for year in years:
    title = 'Average Benefit for Participants by AGI Percentile for {}'.format(year)
    data = var_dict[year]
    data_dict = {'index': [i for i in range(1, 101)]}
    for ben in ben_list:
        ben_name = '{}_ben'.format(ben)
        sub_data = copy.deepcopy(data[data[ben_name] > 0])
        # find the average benefits
        ben_avg = percentile(sub_data, ben_name, 100, 'c00100')
        data_dict[ben_name_map[ben]] = ben_avg
    # create CDS
    cds = ColumnDataSource(data_dict)
    # create plot
    plot = avg_ben(title, cds)
    plot.y_range = Range1d(0, 45000)
    avg_part_list.append(plot)

In [19]:
show(column(avg_part_list))

In [20]:
def participation_rates(data, year, ben):
    """
    """
    f = figure(title='{}, {}'.format(ben, year),
               width=300, height=300)
    f.line(x='index', y='rate', line_width=2, source=data)
    hover = HoverTool(tooltips=[('Percentile', '@index'), ('Rate', '@rate')])
    f.add_tools(hover)
    return f

In [21]:
part_list = []  # list for each year's participation plots
for year in years:
    # list for all the plots for each year
    part_yr_list = []
    data = var_dict[year]
    for ben in ben_list:
        part_name = '{}_part'.format(ben)
        # find participation rates
        p_rate = percentile(data, part_name, 100, 'c00100')
        # create plot
        p_rate_cds = ColumnDataSource({'rate': p_rate, 'index': [i for i in range(1, 101)]})
        b_name = ben_name_map[ben]
        plot = participation_rates(p_rate_cds, year, b_name)
        part_yr_list.append(plot)
    # split the plots into lists of length 2 to be used in a grid plot
    for i in range(0, len(part_yr_list), 2):
        sliced_list = part_yr_list[i: i + 2]
        part_list.append(sliced_list)

### Participation Rates by AGI Percentile

In [22]:
show(gridplot(part_list))